In [1]:
import numpy as np
import pandas as pd

# === 参数定义 ===
deg = np.pi / 180  # 角度转弧度
S = 1.0  # 自旋大小（单位向量）

# α_n 和 φ_n （每层）单位为度，从 Figure 5(d)
alpha_deg = [31, 44, 31, 44, 31, 44]
phi_deg = [17, 16, 17, 16, 17, 16]
gamma_deg = [0, 0, 0, 0, 0, 0]  # γ_n = 0 表示自旋平面垂直 z 轴

# ξ_n = +1 (偶数层)，-1 (奇数层)
xi_n = [1 if n % 2 == 0 else -1 for n in range(6)]

# === 单位矢量定义 ===
x_hat = np.array([1, 0, 0])
y_hat = np.array([0, 1, 0])
z_hat = np.array([0, 0, 1])

# α_n 平面方向矢量
def e_alpha(alpha_rad):
    return np.cos(alpha_rad) * x_hat + np.sin(alpha_rad) * y_hat

def e_alpha_prime(alpha_rad):
    return np.cos(alpha_rad + np.pi/2) * x_hat + np.sin(alpha_rad + np.pi/2) * y_hat

# === 计算每层 A, B, C 子晶格的磁矩 ===
magnetic_moments = []

for n in range(6):
    alpha_n = alpha_deg[n] * deg
    phi_n = phi_deg[n] * deg
    gamma_n = gamma_deg[n] * deg
    xi = xi_n[n]

    # 方向矢量
    e_a = e_alpha(alpha_n)
    e_a_p = e_alpha_prime(alpha_n)

    # A_n 自旋方向
    S_A = (np.cos(gamma_n) * np.cos(phi_n) * z_hat +
           np.sin(phi_n) * e_a +
           np.sin(gamma_n) * np.cos(phi_n) * e_a_p)

    # B_n 自旋方向：φ + ξ * 120°
    phi_B = phi_n + xi * 2 * np.pi / 3
    S_B = (np.cos(gamma_n) * np.cos(phi_B) * z_hat +
           np.sin(phi_B) * e_a +
           np.sin(gamma_n) * np.cos(phi_B) * e_a_p)

    # C_n 自旋方向：φ - ξ * 120°
    phi_C = phi_n - xi * 2 * np.pi / 3
    S_C = (np.cos(gamma_n) * np.cos(phi_C) * z_hat +
           np.sin(phi_C) * e_a +
           np.sin(gamma_n) * np.cos(phi_C) * e_a_p)

    # 加入列表
    magnetic_moments.append((f"A{n}", *S_A))
    magnetic_moments.append((f"B{n}", *S_B))
    magnetic_moments.append((f"C{n}", *S_C))

# === 构造 DataFrame 展示结果 ===
df_moments = pd.DataFrame(magnetic_moments, columns=["Site", "Mx", "My", "Mz"])
df_moments


,Site,Mx,My,Mz
0,A0,0.250611,0.150583,0.956305
1,B0,0.584587,0.351255,-0.731354
2,C0,-0.835198,-0.501838,-0.224951
3,A1,0.198277,0.191474,0.961262
4,B1,-0.697972,-0.674024,-0.241922
5,C1,0.499695,0.482550,-0.719340
6,A2,0.250611,0.150583,0.956305
7,B2,0.584587,0.351255,-0.731354
8,C2,-0.835198,-0.501838,-0.224951
9,A3,0.198277,0.191474,0.961262


In [2]:
# PdCrO2_RT_neutron_mag.ipynb  ——  2025‑05‑06 修正版
import numpy as np
import pandas as pd
from ase.spacegroup import crystal
from ase.build import make_supercell
from ase.io import write

# -------------------------------------------------------
# 1. 原胞：R‑3m 六方 (3 f.u.) —— neutron (RT) 数据
a  = 2.9280   # Å
c  = 18.1217  # Å
zO = 0.11057  # O 的 z 位置

prim = crystal(
    symbols=['Pd', 'Cr', 'O'],
    basis=[(0, 0, 0),          # Pd 3a
           (0, 0, 0.5),        # Cr 3b
           (0, 0, zO)],        # O  6c
    spacegroup=166,
    cellpar=[a, a, c, 90, 90, 120],
    primitive_cell=False
)

# -------------------------------------------------------
# 2. 转换矩阵：a_M = 2a+b,  b_M = a+2b,  c_M = 2c
T = np.array([[2, 1, 0],
              [1, 2, 0],
              [0, 0, 2]], dtype=int)

magcell = make_supercell(prim, T)   # 72 原子 = 18 Pd + 18 Cr + 36 O

# -------------------------------------------------------
# 3. 生成 Cr 的 A0…C5 标签（螺旋 ABC）
cr_idx = [i for i, s in enumerate(magcell) if s.symbol == 'Cr']
frac_all = magcell.get_scaled_positions()              # 72×3
frac_cr  = frac_all[cr_idx]                            # 18×3

# (a) 按 z 坐标升序得到 6 层
layer_order = [idx for idx, _z in sorted(zip(cr_idx, frac_cr[:, 2]),
                                         key=lambda t: t[1])]
layers = [layer_order[i:i+3] for i in range(0, 18, 3)]   # 6×3

# (b) 每层几何 A′/B′/C′  → 螺旋 ABC
ideal_xy = np.array([[0, 0],
                     [2/3, 1/3],
                     [1/3, 2/3]])
labels = {}
for L, layer in enumerate(layers):          # L = 0…5
    xy = frac_all[layer][:, :2] % 1.0
    dist = np.linalg.norm(xy[:, None] - ideal_xy[None], axis=2)
    geo_order = dist.argmin(1)              # 0→A′,1→B′,2→C′

    shift = L % 3                           # 螺旋偏移
    mag_order = (geo_order + shift) % 3     # 0→A,1→B,2→C

    for idx, sub in zip(layer, mag_order):
        labels[idx] = f"{'ABC'[sub]}{L}"

# (c) 写进 magcell.arrays
all_labels = np.array([""] * len(magcell), dtype=object)
for idx, lab in labels.items():
    all_labels[idx] = lab
magcell.new_array("mag_label", all_labels)

# -------------------------------------------------------
# 4. 写 POSCAR（不含磁矩）
write("PdCrO2_RT_neutron_mag.vasp", magcell, vasp5=True, direct=True)
print("POSCAR 已写出：PdCrO2_RT_neutron_mag.vasp\n")

# -------------------------------------------------------
# 5. Cr 对照表
cr_labels = [labels[i] for i in cr_idx]     # ← 供打印与磁矩查找
table = pd.DataFrame({
    "ASE_index": cr_idx,
    "Label": cr_labels,
    "x_frac": frac_all[cr_idx, 0],
    "y_frac": frac_all[cr_idx, 1],
    "z_frac": frac_all[cr_idx, 2]
}).sort_values("ASE_index").reset_index(drop=True)

print("Cr 原子索引 ↔ 论文编号：")
display(table)

# -------------------------------------------------------
# 6. 统计
nPd = (np.array(magcell.get_chemical_symbols()) == 'Pd').sum()
nCr = len(cr_idx)
nO  = (np.array(magcell.get_chemical_symbols()) == 'O').sum()
print(f"\n总原子数 {len(magcell)}  (Pd {nPd}, Cr {nCr}, O {nO})")

# =============================================
# 7. 给每个 Cr 分配磁矩向量 (非共线)
# =============================================
moments = {
    #        Mx        My        Mz
    "A0": ( 0.250611,  0.150583,  0.956305),
    "A1": ( 0.198277,  0.191474,  0.961262),
    "A2": ( 0.250611,  0.150583,  0.956305),
    "A3": ( 0.198277,  0.191474,  0.961262),
    "A4": ( 0.250611,  0.150583,  0.956305),
    "A5": ( 0.198277,  0.191474,  0.961262),

    "B0": ( 0.584587,  0.351255, -0.731354),
    "B1": (-0.697972, -0.674024, -0.241922),
    "B2": ( 0.584587,  0.351255, -0.731354),
    "B3": (-0.697972, -0.674024, -0.241922),
    "B4": ( 0.584587,  0.351255, -0.731354),
    "B5": (-0.697972, -0.674024, -0.241922),

    "C0": (-0.835198, -0.501838, -0.224951),
    "C1": ( 0.499695,  0.482550, -0.719340),
    "C2": (-0.835198, -0.501838, -0.224951),
    "C3": ( 0.499695,  0.482550, -0.719340),
    "C4": (-0.835198, -0.501838, -0.224951),
    "C5": ( 0.499695,  0.482550, -0.719340),
}
magmom = np.zeros((len(magcell), 3))
for idx in cr_idx:
    magmom[idx] = moments[magcell.arrays["mag_label"][idx]]
magcell.set_initial_magnetic_moments(magmom)

# 打印 INCAR 的 MAGMOM 行
triplets = ["{:+.6f} {:+.6f} {:+.6f}".format(*m) for m in magmom]
print("\n-------- 复制以下内容到 INCAR ---------------")
print("MAGMOM = " + "  ".join(triplets))
print("------------------------------------------------\n")

# 带磁矩、标签的 POSCAR
write("PdCrO2_RT_neutron_mag_withMom.vasp", magcell, vasp5=True, direct=True)
print("已写出 PdCrO2_RT_neutron_mag_withMom.vasp\n")

# 额外检查
print("len(magmom) =", len(magmom))
print("非零向量数 =", (np.linalg.norm(magmom, axis=1) != 0).sum())

# 纯数字文本
np.savetxt("MAGMOM.txt", magmom, fmt="%+.6f")
print("已写出 → MAGMOM.txt （72×3）")


POSCAR 已写出：PdCrO2_RT_neutron_mag.vasp

Cr 原子索引 ↔ 论文编号：


,ASE_index,Label,x_frac,y_frac,z_frac
0,3,B1,0.000000e+00,0.000000e+00,0.250000
1,4,C2,3.333333e-01,0.000000e+00,0.416667
2,5,A0,1.134482e-16,3.333333e-01,0.083333
3,15,B4,0.000000e+00,0.000000e+00,0.750000
4,16,C5,3.333333e-01,0.000000e+00,0.916667
5,17,A3,1.134482e-16,3.333333e-01,0.583333
6,27,C1,3.333333e-01,3.333333e-01,0.250000
7,28,A2,6.666667e-01,3.333333e-01,0.416667
8,29,C0,3.333333e-01,6.666667e-01,0.083333
9,39,C4,3.333333e-01,3.333333e-01,0.750000



总原子数 72  (Pd 18, Cr 18, O 36)

-------- 复制以下内容到 INCAR ---------------
MAGMOM = +0.000000 +0.000000 +0.000000  +0.000000 +0.000000 +0.000000  +0.000000 +0.000000 +0.000000  -0.697972 -0.674024 -0.241922  -0.835198 -0.501838 -0.224951  +0.250611 +0.150583 +0.956305  +0.000000 +0.000000 +0.000000  +0.000000 +0.000000 +0.000000  +0.000000 +0.000000 +0.000000  +0.000000 +0.000000 +0.000000  +0.000000 +0.000000 +0.000000  +0.000000 +0.000000 +0.000000  +0.000000 +0.000000 +0.000000  +0.000000 +0.000000 +0.000000  +0.000000 +0.000000 +0.000000  +0.584587 +0.351255 -0.731354  +0.499695 +0.482550 -0.719340  +0.198277 +0.191474 +0.961262  +0.000000 +0.000000 +0.000000  +0.000000 +0.000000 +0.000000  +0.000000 +0.000000 +0.000000  +0.000000 +0.000000 +0.000000  +0.000000 +0.000000 +0.000000  +0.000000 +0.000000 +0.000000  +0.000000 +0.000000 +0.000000  +0.000000 +0.000000 +0.000000  +0.000000 +0.000000 +0.000000  +0.499695 +0.482550 -0.719340  +0.250611 +0.150583 +0.956305  -0.835198 -0.501838 -

In [3]:
import argparse, numpy as np
from pymatgen.core import Structure

parser = argparse.ArgumentParser()
parser.add_argument("-m", "--magmom", default="MAGMOM.txt")
parser.add_argument("-p", "--poscar", default="POSCAR")
parser.add_argument("-o", "--mcif",   default="vesta.mcif")
args, _ = parser.parse_known_args()          # ← 让 notebook 忽略 -f …

# --- 读取 POSCAR ---
structure = Structure.from_file(args.poscar)

# --- 读取 MAGMOM ---
mag_flat = np.fromstring(open(args.magmom).read(), sep=" ")
magmoms  = mag_flat.reshape(-1, 3)

if len(structure) != len(magmoms):
    raise ValueError(f"Atom count mismatch: POSCAR {len(structure)}, MAGMOM {len(magmoms)}")

for i, site in enumerate(structure):
    site.properties["magmom"] = magmoms[i]

structure.to(fmt="mcif", filename=args.mcif)
print("MCIF written ➜", args.mcif)


MCIF written ➜ vesta.mcif
